In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [2]:
autoencoder_model = tf.keras.models.load_model('/content/drive/MyDrive/old_model/autoencoder_classification.h5')

In [3]:
!mkdir -p /drive/ngrok-ssh
%cd /drive/ngrok-ssh
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!cp /drive/ngrok-ssh/ngrok /ngrok
!chmod +x /ngrok

/drive/ngrok-ssh
--2022-07-07 11:44:44--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 54.161.241.46, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  60.9MB/s    in 0.2s    

2022-07-07 11:44:44 (60.9 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [4]:
!pip install flask_ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import pandas as pd
from flask_ngrok import run_with_ngrok
from flask import request, jsonify, Flask
import random as rk

In [6]:
from pathlib import Path

In [7]:
import logging
logging.basicConfig(filename="/content/drive/MyDrive/old_model/log_file.log",
                            filemode='a',
                            format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                            datefmt='%Y-%m-%d %H:%M:%S',
                            level=logging.DEBUG)

In [8]:
!/ngrok authtoken 27ajBoeqpt13OHppl7qZNCtMOwj_45butwz4Dr69pieqRAeyG

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [9]:
def my_round(arr, N):
  new_arr = []
  for i in arr:
    new_arr.append(format(i, '.3g'))
  return new_arr

In [10]:
def prepareResponse(classification_probabilities, predicted_classes_rounded):
  json_data = {}
  classification_result = {}
  probabilities = {}
  indx = classification_probabilities.index(max(classification_probabilities))
  if predicted_classes_rounded == 0:
    json_data['classification_result']="DoS attack"
  elif predicted_classes_rounded == 1:
    json_data['classification_result']="Normal"
  elif predicted_classes_rounded == 2:
    json_data['classification_result']="Probe attack"
  elif predicted_classes_rounded == 3:
    json_data['classification_result']="R2L attack"
  elif predicted_classes_rounded == 4:
    json_data['classification_result']="U2R attack"
  probabilities['DOS']=classification_probabilities[0]
  probabilities['Normal']=classification_probabilities[1]
  probabilities['Probe']=classification_probabilities[2]
  probabilities['R2L']=classification_probabilities[3]
  probabilities['U2R']=classification_probabilities[4]
  json_data['probabilities']=probabilities
  return json_data

In [11]:
app = Flask(__name__)
run_with_ngrok(app)

In [12]:
@app.route("/") 

def home():
  return "</h1> DCAE-CIDS | Nujitha </h1>"

In [13]:
@app.route('/consult', methods=['GET','POST'])

def consult():
 res = {}
 try:
    payload_req = request.json
    try:
      predicted_classes = autoencoder_model.predict(payload_req['feedback'])
      predicted_classes_rounded = np.argmax(np.round(predicted_classes),axis=1)
      res = prepareResponse(my_round(predicted_classes[0], 2), predicted_classes_rounded)
      req_str = ''.join([str(elem) for elem in payload_req['feedback']])
      if res['classification_result'] == "Normal":
        logging.info('Classification result : '+res['classification_result']+' - Feedback request : '+req_str)
      else:
        logging.warning('Classification result : '+res['classification_result']+' - Feedback request : '+req_str)
      return jsonify(res)
    except:
      res['Error']="Internal server error. Please contact admin."
      return jsonify(res), 500
 except:
    res['Error']="Invalid request. Please check request body."
    return jsonify(res), 400

In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://244e-34-86-238-174.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
